In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from numpy import mean, std
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split, LeaveOneOut
from rfpimp import * # might not be installed already, otherwise install using pip install
import collections
from sklearn.inspection import permutation_importance

In [2]:
# load data
data = pd.read_excel('model_data_S_not_zero_absERROR_NotVent.xlsx').drop(columns = ['Unnamed: 0', 'abs_error_estimate'])
data

,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,lh_inferiorparietal_volume,lh_inferiortemporal_volume,lh_isthmuscingulate_volume,lh_lateraloccipital_volume,...,CC_Anterior,age_mri,precuneus_volume,-VentralDC,pericalcarine_volume,-Hippocampus,Putamen,Caudate,S,gender_F
0,0.002964,0.002256,0.008136,0.003394,0.002366,0.010357,0.015142,0.011369,0.003066,0.012157,...,0.000750,19,0.010798,0.004498,0.002535,0.004372,0.005117,0.004709,0.071429,1
1,0.002344,0.001850,0.005794,0.003879,0.002107,0.009607,0.012305,0.009720,0.002845,0.013617,...,0.000802,23,0.009305,0.003978,0.002612,0.003825,0.004903,0.003575,0.468056,1
2,0.002534,0.002797,0.006291,0.002592,0.002028,0.008790,0.012013,0.012749,0.002957,0.012521,...,0.000915,19,0.010651,0.004404,0.001743,0.004377,0.005053,0.003914,0.370714,1
3,0.003097,0.001501,0.006358,0.003707,0.001096,0.009258,0.013345,0.010357,0.002457,0.015477,...,0.000893,18,0.010617,0.003842,0.002555,0.003565,0.005305,0.003608,0.047222,0
4,0.001972,0.002460,0.007522,0.003739,0.001428,0.009609,0.010374,0.008492,0.002886,0.013569,...,0.000869,18,0.010552,0.003802,0.003304,0.003456,0.004619,0.003857,0.400928,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0.002753,0.002395,0.005985,0.002600,0.001503,0.010023,0.012704,0.012537,0.002785,0.013129,...,0.001073,19,0.009620,0.004608,0.002107,0.004487,0.006393,0.004437,0.190540,1
137,0.002515,0.001864,0.007071,0.003184,0.002057,0.009190,0.013648,0.012823,0.003270,0.010043,...,0.000735,22,0.011233,0.003766,0.002369,0.003947,0.004982,0.003704,0.227739,0
138,0.002578,0.001552,0.007039,0.003653,0.002337,0.010593,0.012364,0.010617,0.002702,0.014111,...,0.000791,22,0.010226,0.004725,0.002690,0.004314,0.005037,0.004465,0.314423,0
139,0.002340,0.001347,0.007087,0.002897,0.002138,0.009919,0.014293,0.011200,0.002929,0.012897,...,0.000938,20,0.010535,0.003965,0.002674,0.003505,0.004694,0.003493,0.213333,0


In [7]:
# prep data into X and Y (X also scaled for the linear model)
X = data.drop(columns = ['S'])
X_scaled = MinMaxScaler().fit_transform(X) # REMARK -> transforming data here makes the model less generalizable
Y = data.S
print( X.shape, Y.shape)

(141, 85) (141,)


In [8]:
# create a new set of random values for the RANDOM feature for each new cv
X_sc_df = pd.DataFrame(X_scaled, columns = X.columns)
X_sc_df.loc[:, ('RANDOM')] = np.random.uniform(0,1, size = len(X_sc_df))

**LOOCV LASSO REGRESSION**

In [9]:
# initiate the storage for the data
MSE_lasso = list()
MSE_base = list()
Y_preds = list()
Y_trues = list()
Y_means = list()
coef = collections.defaultdict(list)
alphas = list()

# prep loocv
cv = LeaveOneOut()
cv.get_n_splits(X_sc_df)

# make feature list
feature_list = X_sc_df.keys().tolist()

# execute the loocv
for train_ix, test_ix in cv.split(X_sc_df):
    
    # create a new set of random values for the RANDOM feature for each new cv
    X_sc_df.loc[:, ('RANDOM')] = np.random.uniform(0,1, size = len(X_sc_df)) 
    
    # split data
    X_train, X_test = X_sc_df.iloc[train_ix, :], X_sc_df.iloc[test_ix, :]
    y_train, y_test = Y.iloc[train_ix], Y.iloc[test_ix]

    # create the model grid search for finetuning alpha (the inner loop)
    cv_inner = KFold(n_splits = 5, shuffle = True, random_state = 10)
    
    # define model
    model_s = linear_model.Lasso()
    
    # define grid
    grid = {"alpha": np.arange(0.001, 0.999, 0.001)} 

    # fit the gridsearch
    search = GridSearchCV(model_s, grid, scoring = 'neg_mean_squared_error', cv = cv_inner, n_jobs = 2)
    search.fit(X_train, y_train)
    
    # get best model from cross validated grid search 
    model = search.best_estimator_
    alphas.append(model.alpha) # store alpha value resulting from the grid search
    
    # evaluate best Lasso model
    y_pred = model.predict(X_test)
    Y_preds.append(y_pred) # store predicted Y value
    score = mean_squared_error(y_test, y_pred)
    MSE_lasso.append(score) # store Lasso model score
    
    # store true Y value
    Y_trues.append(y_test)
    
    # evaluate baseline
    y_pred_base = [mean(y_train)]
    Y_means.append(y_pred_base) # store base model Y value
    score_base = mean_squared_error(y_test, y_pred_base)
    MSE_base.append(score_base) # store base model score
    
    print(model)
    
    # store coefficients of the features
    coe = model.coef_
    for feature, coeff in zip(feature_list, coe):
        coef[feature].append(coeff)
        
# print mean MSE and its standarddeviation once finished the loocv
print('MSE Lasso: %.3f (%.3f)' % (mean(MSE_lasso), std(MSE_lasso)))
print('MSE Baseline: %.3f (%.3f)' % (mean(MSE_base), std(MSE_base)))

Lasso(alpha=0.008)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.008)
Lasso(alpha=0.008)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.004)
Lasso(alpha=0.011)
Lasso(alpha=0.011)
Lasso(alpha=0.011)
Lasso(alpha=0.011)
Lasso(alpha=0.010000000000000002)
Lasso(alpha=0.003)
Lasso(alpha=0.004)
Lasso(alpha=0.003)
Lasso(alpha=0.011)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.011)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.004)
Lasso(alpha=0.003)
Lasso(alpha=0.003)
Lasso(alpha=0.003)
Lasso(alpha=0.003)
Lasso(alpha=0.003)
Lasso(alpha=0.004)
Lasso(alpha=0.003)
Lasso(alpha=0.003)
Lasso(alpha=0.003)

In [10]:
# calculate mean and std for coefficients and save coefficients to excel
coef_lasso = pd.DataFrame.from_dict(coef, orient = 'index')
coef_lasso_tot = coef_lasso.copy()
coef_lasso_tot['coefficient'] = coef_lasso.mean(axis = 1)
coef_lasso_tot['std'] = coef_lasso.std(axis = 1)
coef_lasso_tot.to_excel("coef_lasso_loocv.xlsx")

In [11]:
# calculate mean and std of MSE of the lasso and Base model and save MSE to excel
MSE = dict()
MSE['lasso'] = MSE_lasso
MSE['base'] = MSE_base
MSE = pd.DataFrame.from_dict(MSE, orient = 'index')
MSE_tot = MSE.copy()
MSE_tot['mean'] = MSE.mean(axis = 1)
MSE_tot['std'] = MSE.std(axis = 1)
MSE_tot.to_excel("MSE_lasso_loocv.xlsx")

In [12]:
# combine y_pred, y_mean (base prediction), y_true and alpha and save to excel
S_values = dict()
S_values['y_pred'] = [item[0] for item in Y_preds]
S_values['y_mean'] = [item[0] for item in Y_means]
S_values['y_true'] = [float(item) for item in Y_trues]
S_values['alpha'] = alphas
S_value = pd.DataFrame.from_dict(S_values, orient = 'index')
S_value.to_excel("S_values_lasso_loocv.xlsx")

**LOOCV RANDOM FOREST REGRESSION**

In [ ]:
# initiate storage of the data
rf_feature_importance_perm = collections.defaultdict(list) # permutation of feature importances
rf_feature_importance_perm_std = collections.defaultdict(list) # permutation of feature importances
Y_preds_rf = list()
Y_trues_rf = list()
Y_means_rf = list()
MSE_rf = list()
MSE_base_rf = list()
count = 0

# prepare loocv
cv_outer = LeaveOneOut()
cv_outer.get_n_splits(X)

# execute the loocv 
for train_ix, test_ix in cv_outer.split(X):
    
    # create a new set of random values for the RANDOM feature for each new cv
    X.loc[:, ('RANDOM')] = np.random.uniform(0,1, size = len(X)) 
    
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
    # initiate and fit the model
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs = 2)
    rf.fit(X_train, y_train)
    print("fitted model:", count)

    # evaluate best RF model
    y_pred = rf.predict(X_test)
    Y_preds_rf.append(y_pred) # store predicted Y value
    score = mean_squared_error(y_test, y_pred)
    MSE_rf.append(score) # store RF model score
    
    # evaluate base model
    y_base = [mean(y_train)]
    Y_means_rf.append(y_base) # store base model Y value
    score_base = mean_squared_error(y_test, y_base)
    MSE_base_rf.append(score_base) # store base model score
    
    # store true Y value
    Y_trues_rf.append(y_test) 
    
    # initiate a list of all features
    feature_list = X_train.keys().tolist()

    # also store permutated feature importances and its standarddeviation
    imp = permutation_importance(rf, X_train, y_train, scoring = 'neg_mean_squared_error', random_state = 0,  n_jobs = 2)
    for item in imp:
        if item == 'importances_mean':
            for feature, importance in zip(feature_list, imp[item]):
                rf_feature_importance_perm[feature].append(importance)       
        if item == 'importances_std':
            for feature, importance_std in zip(feature_list, imp[item]):
                rf_feature_importance_perm_std[feature].append(importance_std)
        
    # increase count
    count = count + 1

# print mean MSE and its standarddeviation once finished the loocv
print('MSE RF: %.3f (%.3f)' % (mean(MSE_rf), std(MSE_rf)))
print('MSE Baseline: %.3f (%.3f)' % (mean(MSE_base), std(MSE_base)))

fitted model: 0
fitted model: 1
fitted model: 2
fitted model: 3
fitted model: 4
fitted model: 5
fitted model: 6
fitted model: 7
fitted model: 8
fitted model: 9
fitted model: 10
fitted model: 11
fitted model: 12
fitted model: 13
fitted model: 14
fitted model: 15
fitted model: 16
fitted model: 17
fitted model: 18
fitted model: 19
fitted model: 20
fitted model: 21
fitted model: 22
fitted model: 23
fitted model: 24
fitted model: 25
fitted model: 26
fitted model: 27
fitted model: 28
fitted model: 29
fitted model: 30
fitted model: 31
fitted model: 32
fitted model: 33
fitted model: 34
fitted model: 35
fitted model: 36
fitted model: 37
fitted model: 38
fitted model: 39
fitted model: 40
fitted model: 41
fitted model: 42
fitted model: 43
fitted model: 44
fitted model: 45
fitted model: 46
fitted model: 47
fitted model: 48
fitted model: 49
fitted model: 50
fitted model: 51
fitted model: 52
fitted model: 53
fitted model: 54
fitted model: 55
fitted model: 56
fitted model: 57
fitted model: 58
fitted 

In [ ]:
# calculate mean and std of permutation importance and save to excel
rf_fi_perm = pd.DataFrame.from_dict(rf_feature_importance_perm, orient = 'index')
RF_perm = rf_fi_perm.copy()
RF_perm['importance'] = rf_fi_perm.mean(axis = 1)
RF_perm['std'] = rf_fi_perm.std(axis = 1)
RF_perm.to_excel("RF_LOOCV_perm_importance.xlsx")

In [ ]:
# calculate mean and std of MSE of the rf and Base model and save MSE to excel
MSE = dict()
MSE['RF'] = MSE_rf
MSE['base'] = MSE_base_rf
MSE = pd.DataFrame.from_dict(MSE, orient = 'index')
MSE_tot = MSE.copy()
MSE_tot['mean'] = MSE.mean(axis = 1)
MSE_tot['std'] = MSE.std(axis = 1)
MSE_tot.to_excel("MSE_RF_LOOCV.xlsx")

In [ ]:
# combine y_pred, y_mean (base prediction), y_true and alpha and save to excel
S_values = dict()
S_values['y_pred'] = [item[0] for item in Y_preds_rf]
S_values['y_mean'] = [item[0] for item in Y_means_rf]
S_values['y_true'] = [float(item) for item in Y_trues_rf]
S_value = pd.DataFrame.from_dict(S_values, orient = 'index')
S_value.to_excel("S_values_RF_loocv.xlsx")